> # 공통 환경 처리

In [2]:
%pip install influxdb_client

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import pandas as pd
import glob
from influxdb_client import InfluxDBClient
from datetime import datetime, timedelta

### Influx DB 접속 정보 및 장치 위치 맵핑 정보

In [ ]:
# InfluxDB 설정 정보
url = "http://133.186.144.22:8086"
token = "BPJ1pnKvoaov4Tte971t0zpRSTUXNZvrshU7u3UPheAIsBeUJEFfbKjfsZjtwZmugkHJEGRW17lH4bR9ybanNQ=="
org = "smoothing"

# 디바이스 ID와 위치를 매핑
location_mapping = {
    '24e124126d152919': 'indoor',
    '24e124126d152969': 'bottom_right_corner',
    '24e124128c067999': 'indoor',
    '24e124785c389818': 'bottom_left_corner',
    '24e124785c421885': 'top_right_corner'
}

> # 데이터 집계 함수

influx DB에서 데이터를 받아와 Time, Value, Place, Location, Device 컬럼 형태로 지정을 하였다.

또한 전력 데이터와 온도 데이터를 받을 예정이다.

In [4]:
# InfluxDB 클라이언트 생성
def create_client(url, token, org):
    return InfluxDBClient(url=url, token=token, org=org)

# 쿼리 실행 및 DataFrame으로 변환
def query_to_dataframe(client, query):
    result = client.query_api().query(query=query)
    results = []
    
    for table in result:
        for record in table.records:
            results.append({
                "time": record.get_time(),
                "value": record.get_value(),
                "place": record.values.get("place"),
                "location": record.values.get("location"),
                "device": record.values.get("device")
            })
    
    df = pd.DataFrame(results)
    df['time'] = df['time'].astype(str).str.replace(r'\+00:00$', '', regex=True)
    return df

# 데이터를 날짜를 지정하여 CSV 파일로 저장
def save_csv(df, file_pattern, directory):
    # 경로가 존재하는지 확인하고, 없다면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    current_date = datetime.now()
    previous_date = current_date - timedelta(days=1)
    filename = f"{directory}{previous_date.strftime(file_pattern)}"
    df.to_csv(filename, index=False)
    
# 온도 Data에서 'device' 열에 따라 'location' 열을 업데이트    
def update_location(df, location_mapping):
    df['location'] = df['device'].map(location_mapping)
    return df

### 각 데이터 조회 및 CSV변환 수행

In [5]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 전력 조회 Flux 쿼리
query_powermetrics = '''
import "date"

from(bucket: "powermetrics_data")
  |> range(start: date.sub(d: 1d, from: date.truncate(t: now(), unit: 1d)), stop: date.truncate(t: now(), unit: 1d))
  |> filter(fn: (r) => r["phase"] == "total")
  |> filter(fn: (r) => r["description"] == "w")
  |> filter(fn: (r) => r["place"] == "class_a" or r["place"] == "office")
  |> filter(fn: (r) => r["location"] != "main")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# CSV 변환
df_powermetrics = query_to_dataframe(client, query_powermetrics)
print(df_powermetrics.head())
save_csv(df_powermetrics, "%m_%d_powermetrics_data.csv", "powermetrics/")


# 온도 조회 Flux 쿼리
query_environmental = '''
import "date"
from(bucket: "environmentalsensors_data")
  |> range(start: date.sub(d: 1d, from: date.truncate(t: now(), unit: 1d)), stop: date.truncate(t: now(), unit: 1d))
  |> filter(fn: (r) => r["measurement"] == "temperature")
  |> filter(fn: (r) => r["place"] == "class_a" or r["place"] == "office")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# CSV 변환
df_environmental = query_to_dataframe(client, query_environmental)
df_environmental_fix = update_location(df_environmental, location_mapping)
print(df_environmental_fix.head())
save_csv(df_environmental_fix, "%m_%d_environmentalsensors_data.csv", "environmentalsensors/")

# 클라이언트 종료
client.close()

                  time  value   place   location     device
0  2024-04-23 00:02:00   26.0  office  a_project  gems-3500
1  2024-04-23 00:04:00   25.0  office  a_project  gems-3500
2  2024-04-23 00:06:00   26.0  office  a_project  gems-3500
3  2024-04-23 00:08:00   25.0  office  a_project  gems-3500
4  2024-04-23 00:10:00   26.0  office  a_project  gems-3500
                  time  value   place location            device
0  2024-04-23 00:02:00   23.6  office   indoor  24e124126d152919
1  2024-04-23 00:04:00   23.5  office   indoor  24e124126d152919
2  2024-04-23 00:06:00   23.6  office   indoor  24e124126d152919
3  2024-04-23 00:08:00   23.6  office   indoor  24e124126d152919
4  2024-04-23 00:10:00   23.6  office   indoor  24e124126d152919


> # 데이터 전처리

데이터 전처리를 위해 결측치 제거와 이상치 제거를 할 것이다.

온도데이터와, 전력데이터 빈값이 있을 수 있으므로 결측치를 제거한다.

이상치는 어떠한 것을 위해 제거한다.

### 전체 데이터 병합 및 결측치 처리 함수

In [25]:
# 전체 데이터 병합
def merge_and_compute_stats_by_time_and_place(directory_path, file_name):
    # 디렉토리 내의 모든 CSV 파일 목록을 생성
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

    # 모든 CSV 파일을 데이터 프레임으로 읽어와 하나로 병합
    data_frames = []
    for file in csv_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path)
        # 'time' 컬럼을 datetime으로 변환
        df['time'] = pd.to_datetime(df['time'])
        # 'time_hour_min' 컬럼을 생성하여 시간과 분만 추출
        df['time_hour_min'] = df['time'].dt.strftime('%H:%M')
        data_frames.append(df)

    # 모든 데이터 프레임을 하나로 병합
    merged_df = pd.concat(data_frames, ignore_index=True)

    # 'time', 'place', 'location' 별로 그룹화하고 각 통계치 계산
    result_df = merged_df.groupby(['time_hour_min', 'place', 'location'])['value'].agg(
        min_value='min',       # 최소값
        max_value='max',       # 최대값
        mean_value='mean',     # 평균
        median_value='median', # 중앙값
    ).reset_index()

    # std_dev 컬럼에서 NaN 값 0으로 대체
    result_df['std_dev'].fillna(0, inplace=True)

    # 결과 데이터 프레임을 CSV 파일로 저장
    result_df.to_csv(file_name, index=False)
    
    return result_df

# 결측치 확인 및 처리
def load_and_clean_data(input_directory_path, output_directory_path):
    summary = []

    # 해당 디렉토리의 모든 파일을 리스트업
    files = [f for f in os.listdir(input_directory_path) if f.endswith('.csv')]

    for file in files:
        file_summary = {"file": file}
        input_file_path = os.path.join(input_directory_path, file)

        # 데이터 불러오기
        df = pd.read_csv(input_file_path)
        initial_shape = df.shape

        # 결측치 제거
        missing_before = df.isnull().sum().sum()
        df.dropna(inplace=True)
        missing_after = df.isnull().sum().sum()

        # 결과 요약
        file_summary["initial_rows"] = initial_shape[0]
        file_summary["final_rows"] = df.shape[0]
        file_summary["missing_removed"] = missing_before

        # 수정된 데이터 저장
        if not os.path.exists(output_directory_path):
            os.makedirs(output_directory_path)
        output_file_path = os.path.join(output_directory_path, f"{os.path.splitext(file)[0]}_cleaned.csv")
        df.to_csv(output_file_path, index=False)

        summary.append(file_summary)

    # 결과 요약 출력
    print("\n--- 처리 결과 요약 ---\n")
    for s in summary:
        print(f"파일명: {s['file']}, 초기 행 수: {s['initial_rows']}, 최종 행 수: {s['final_rows']}, 제거된 결측치 수: {s['missing_removed']}")

### 결측치 확인 및 처리 수행

In [29]:
input_directory_path = 'powermetrics/'
output_directory_path = 'cleaned_data/powermetrics/'
load_and_clean_data(input_directory_path, output_directory_path)

input_directory_path = 'environmentalsensors/'
output_directory_path = 'cleaned_data/environmentalsensors/'
load_and_clean_data(input_directory_path, output_directory_path)


--- 처리 결과 요약 ---

파일명: 04_21_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_15_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_16_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_17_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_20_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_18_powermetrics_data.csv, 초기 행 수: 19197, 최종 행 수: 19197, 제거된 결측치 수: 0
파일명: 04_23_powermetrics_data.csv, 초기 행 수: 19436, 최종 행 수: 19436, 제거된 결측치 수: 0
파일명: 04_19_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0
파일명: 04_22_powermetrics_data.csv, 초기 행 수: 19440, 최종 행 수: 19440, 제거된 결측치 수: 0

--- 처리 결과 요약 ---

파일명: 04_20_environmentalsensors_data.csv, 초기 행 수: 2853, 최종 행 수: 2853, 제거된 결측치 수: 0
파일명: 04_18_environmentalsensors_data.csv, 초기 행 수: 2817, 최종 행 수: 2817, 제거된 결측치 수: 0
파일명: 04_16_environmentalsensors_data.csv, 초기 행 수: 3316, 최종 행 수: 3316, 제거된 결측치 수: 0
파일명: 04_17_environme

### 각 데이터 병합 수행

In [32]:
powermetrics_path = 'cleaned_data/powermetrics'
file_name = 'powermetrics_stats.csv'
stats_values_df = merge_and_compute_stats_by_time_and_place(powermetrics_path, file_name)
print(stats_values_df.head())

environmentalsensors_path = 'cleaned_data/environmentalsensors'
file_name = 'environmentalsensors_stats.csv'
stats_values_df = merge_and_compute_stats_by_time_and_place(environmentalsensors_path, file_name)
print(stats_values_df.head())

  time_hour_min    place                 location  min_value  max_value  \
0         00:00  class_a           ac_indoor_unit       22.5      166.5   
1         00:00  class_a          ac_outdoor_unit       24.0     4748.0   
2         00:00  class_a           automatic_door        7.0       14.0   
3         00:00  class_a  outdoor_unit_room_light        0.0        0.0   
4         00:00   office                a_project       24.5       28.0   

   mean_value  median_value      std_dev  
0   49.111111          24.0    46.553583  
1  959.722222         150.0  1574.839181  
2    9.277778           8.0     2.359967  
3    0.000000           0.0     0.000000  
4   25.833333          26.0     1.000000  
  time_hour_min    place             location  min_value  max_value  \
0         00:00  class_a   bottom_left_corner       23.0      23.00   
1         00:00  class_a  bottom_right_corner       21.6      23.40   
2         00:00  class_a               indoor       23.0      24.50   
3      

> ### 전력, 온도 데이터의 연관 관계 측정

전력과 온도 데이터를 가지고 연관관계를 측정 해볼 것이다. 

조건은 평균 전력 사용량과 평균 온도를 계산하며 두 데이터 프레임을 시간대 기준으로 결합을 할 것이다. 이후 상관관계에 대해서 수치를 나타내어 확인을 해볼 것이다.

In [31]:
# 전력 사용량 데이터와 환경 센서 데이터를 로드합니다.
powermetrics_df = pd.read_csv('powermetrics_stats.csv')
environmentalsensors_df = pd.read_csv('environmentalsensors_stats.csv')

# 시간대별로 그룹화하여 각 시간대의 평균 전력 사용량을 계산합니다.
power_avg = powermetrics_df.groupby('time_hour_min').mean_value.mean().reset_index(name='avg_power_usage')

# 시간대별로 그룹화하여 각 시간대의 평균 온도를 계산합니다.
temp_avg = environmentalsensors_df.groupby('time_hour_min').mean_value.mean().reset_index(name='avg_temperature')

# 두 데이터프레임을 시간대(time_hour_min)를 기준으로 결합합니다.
combined_data = pd.merge(power_avg, temp_avg, on='time_hour_min')

# 상관관계 계산
correlation = combined_data[['avg_power_usage', 'avg_temperature']].corr()

# 상관관계 출력
print("온도와 전력 사용량 사이의 상관계수는 :", correlation)

온도와 전력 사용량 사이의 상관계수는 :                  avg_power_usage  avg_temperature
avg_power_usage         1.000000         0.892986
avg_temperature         0.892986         1.000000
